In [1]:
import pandas as pd
import numpy as np

import sqlite3
from sqlalchemy import create_engine

In [2]:
pd.set_option('max_colwidth', 3000)
pd.set_option('max_columns', 300)

# First glance at the data

In [87]:
calendar_feb = pd.read_csv('calendar.csv')

In [88]:
calendar_feb.head()

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,2577,2019-02-05,f,$125.00,$125.00,3.0,1125.0
1,2577,2019-02-06,f,$125.00,$125.00,3.0,1125.0
2,2577,2019-02-07,f,$125.00,$125.00,3.0,1125.0
3,2577,2019-02-08,f,$125.00,$125.00,3.0,1125.0
4,2577,2019-02-09,f,$125.00,$125.00,3.0,1125.0


In [13]:
calendar_feb.count()

listing_id        21195164
date              21195164
available         21195164
price             21194984
adjusted_price    21194984
minimum_nights    21195147
maximum_nights    21195147
dtype: int64

In [14]:
calendar_feb.date.max()

'2020-02-05'

In [15]:
(
    calendar_feb
    .query('listing_id == 2577')
    .count()
    .head(1)
)

listing_id    365
dtype: int64

# Put the calendar for each month to an SQLite database

because it's much faster to query

In [2]:
engine = create_engine('sqlite:///liligo.db', echo=False)

Some hacking is needed to transform prices like '$1,800' to proper numbers

In [143]:
(
    calendar_feb
    .assign(price_usd = lambda x: x.adjusted_price.fillna('0').apply(lambda y: float(y.replace('$', '').replace(',', ''))))
    .drop(['price', 'adjusted_price'], axis = 1)
    .to_sql('calendar_feb', con=engine, if_exists='replace')
)

In [144]:
del calendar_feb

In [145]:
pd.read_sql_query('''
    select * from calendar_feb limit 5
''', engine)

,index,listing_id,date,available,minimum_nights,maximum_nights,price_usd
0,0,2577,2019-02-05,f,3.0,1125.0,125.0
1,1,2577,2019-02-06,f,3.0,1125.0,125.0
2,2,2577,2019-02-07,f,3.0,1125.0,125.0
3,3,2577,2019-02-08,f,3.0,1125.0,125.0
4,4,2577,2019-02-09,f,3.0,1125.0,125.0


In [146]:
calendar_jan = pd.read_csv('http://data.insideairbnb.com/france/ile-de-france/paris/2019-01-13/data/calendar.csv.gz', compression = 'gzip')

In [147]:
calendar_jan.head()

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,70586,2019-01-14,f,$139.00,$139.00,3.0,365.0
1,2577,2019-01-14,f,$125.00,$125.00,3.0,1125.0
2,2577,2019-01-15,f,$125.00,$125.00,3.0,1125.0
3,2577,2019-01-16,f,$125.00,$125.00,3.0,1125.0
4,2577,2019-01-17,f,$125.00,$125.00,3.0,1125.0


In [148]:
(
    calendar_jan
    .assign(price_usd = lambda x: x.adjusted_price.fillna('0').apply(lambda y: float(y.replace('$', '').replace(',', ''))))
    .drop(['price', 'adjusted_price'], axis = 1)
    .to_sql('calendar_jan', con=engine, if_exists='replace')
)

In [149]:
del calendar_jan

In [150]:
pd.read_sql_query('''
    select count(*) as cnt from calendar_jan
''', engine)

,cnt
0,21301394


In [151]:
calendar_dec = pd.read_csv('http://data.insideairbnb.com/france/ile-de-france/paris/2018-12-07/data/calendar.csv.gz', compression = 'gzip')

In [153]:
calendar_dec.head()

,listing_id,date,available,price
0,79690,2019-12-07,t,$185.00
1,79690,2019-12-06,t,$185.00
2,79690,2019-12-05,t,$185.00
3,79690,2019-12-04,t,$185.00
4,79690,2019-12-03,t,$185.00


In [154]:
(
    calendar_dec
    .assign(price_usd = lambda x: x.price.fillna('0').apply(lambda y: float(y.replace('$', '').replace(',', ''))))
    .drop(['price'], axis = 1)
    .to_sql('calendar_dec', con=engine, if_exists='replace')
)

In [155]:
del calendar_dec

In [3]:
pd.read_sql_query('''
    select count(*) as cnt from calendar_dec
''', engine)

,cnt
0,21856565


In [21]:
pd.read_sql_query('''
    select count(*) as cnt
    from (
        select listing_id, available, count(*) as num_available
        from calendar_dec
        group by listing_id, available
        ) t
    where 1=1
        and available = 't'
        and num_available > 30
''', engine)

,cnt
0,24608


In [4]:
calendar_nov = pd.read_csv('http://data.insideairbnb.com/france/ile-de-france/paris/2018-11-06/data/calendar.csv.gz', compression = 'gzip')

In [5]:
calendar_nov.head()

,listing_id,date,available,price
0,29210657,2019-07-14,t,$50.00
1,29777902,2019-11-06,f,NaN
2,29777902,2019-11-05,f,NaN
3,29777902,2019-11-04,f,NaN
4,29777902,2019-11-03,f,NaN


In [6]:
(
    calendar_nov
    .assign(price_usd = lambda x: x.price.fillna('0').apply(lambda y: float(y.replace('$', '').replace(',', ''))))
    .drop(['price'], axis = 1)
    .to_sql('calendar_nov', con=engine, if_exists='replace')
)

In [7]:
del calendar_nov

In [8]:
pd.read_sql_query('''
    select count(*) as cnt from calendar_nov
''', engine)

,cnt
0,22030305


In [9]:
pd.read_sql_query('''
    select count(*) as cnt
    from (
        select listing_id, available, count(*) as num_available
        from calendar_nov
        group by listing_id, available
        ) t
    where 1=1
        and available = 't'
        and num_available > 30
''', engine)

,cnt
0,25259


In [10]:
calendar_oct = pd.read_csv('http://data.insideairbnb.com/france/ile-de-france/paris/2018-10-09/data/calendar.csv.gz', compression = 'gzip')

In [11]:
calendar_oct.head(1)

,listing_id,date,available,price
0,75922,2019-10-08,t,$200.00


In [13]:
(
    calendar_oct
    .assign(price_usd = lambda x: x.price.fillna('0').apply(lambda y: float(y.replace('$', '').replace(',', ''))))
    .drop(['price'], axis = 1)
    .to_sql('calendar_oct', con=engine, if_exists='replace')
)

In [14]:
del calendar_oct

In [15]:
pd.read_sql_query('''
    select count(*) as cnt from calendar_oct
''', engine)

,cnt
0,22346395


In [31]:
pd.read_sql_query('''
    select count(*) as cnt
    from (
        select listing_id, available, count(*) as num_available
        from calendar_oct
        group by listing_id, available
        ) t
    where 1=1
        and available = 't'
        and num_available > 30
''', engine)

,cnt
0,25877


In [233]:
calendar_sep = pd.read_csv('http://data.insideairbnb.com/france/ile-de-france/paris/2018-09-10/data/calendar.csv.gz', compression = 'gzip')

In [234]:
(
    calendar_sep
    .assign(price_usd = lambda x: x.price.fillna('0').apply(lambda y: float(y.replace('$', '').replace(',', ''))))
    .drop(['price'], axis = 1)
    .to_sql('calendar_sep', con=engine, if_exists='replace')
)

In [235]:
pd.read_sql_query('''
    select count(*) as cnt from calendar_sep
''', engine)

,cnt
0,22879660


In [ ]:
del calendar_sep

In [9]:
calendar_aug = pd.read_csv('http://data.insideairbnb.com/france/ile-de-france/paris/2018-08-13/data/calendar.csv.gz', compression = 'gzip')

In [11]:
(
    calendar_aug
    .assign(price_usd = lambda x: x.price.fillna('0').apply(lambda y: float(y.replace('$', '').replace(',', ''))))
    .drop(['price'], axis = 1)
    .to_sql('calendar_aug', con=engine, if_exists='replace')
)

In [3]:
pd.read_sql_query('''
    select count(*) as cnt from calendar_aug
''', engine)

,cnt
0,24102045


In [13]:
del calendar_aug

In [4]:
calendar_jul = pd.read_csv('http://data.insideairbnb.com/france/ile-de-france/paris/2018-07-08/data/calendar.csv.gz', compression = 'gzip')

In [5]:
(
    calendar_jul
    .assign(price_usd = lambda x: x.price.fillna('0').apply(lambda y: float(y.replace('$', '').replace(',', ''))))
    .drop(['price'], axis = 1)
    .to_sql('calendar_jul', con=engine, if_exists='replace')
)

In [6]:
pd.read_sql_query('''
    select count(*) as cnt from calendar_jul
''', engine)

,cnt
0,22938425


In [7]:
del calendar_jul

In [8]:
calendar_jun = pd.read_csv('http://data.insideairbnb.com/france/ile-de-france/paris/2018-06-07/data/calendar.csv.gz', compression = 'gzip')

In [9]:
(
    calendar_jun
    .assign(price_usd = lambda x: x.price.fillna('0').apply(lambda y: float(y.replace('$', '').replace(',', ''))))
    .drop(['price'], axis = 1)
    .to_sql('calendar_jun', con=engine, if_exists='replace')
)

In [10]:
pd.read_sql_query('''
    select count(*) as cnt from calendar_jun
''', engine)

,cnt
0,22092720


In [11]:
del calendar_jun

# Also store the large Listings dataset from February in a table

In [12]:
listings_big = pd.read_csv('listings 2.csv', low_memory=False)

In [13]:
(
    listings_big
    .assign(price_usd = lambda x: x.price.fillna('0').apply(lambda y: float(y.replace('$', '').replace(',', ''))))
    .assign(cleaning = lambda x: x.cleaning_fee.fillna('0').apply(lambda y: float(y.replace('$', '').replace(',', ''))))
    .drop(['price', 'cleaning_fee'], axis = 1)
    .to_sql('listings', con=engine, if_exists='replace')
)

In [14]:
del listings_big

In [95]:
instant = pd.read_sql_query('select id, instant_bookable from listings', con=engine)

In [161]:
instant.head()

,id,instant_bookable
0,2577,t
1,5396,t
2,7397,f
3,7964,f
4,8522,f


# Create small tables for listing_id and 'instant_bookable' flags for each month

In [162]:
instant.to_sql('instant_feb', con=engine, if_exists='replace')

In [163]:
pd.read_sql_query('select instant_bookable, count(*) as cnt from instant_feb group by instant_bookable', con=engine)

,instant_bookable,cnt
0,f,38526
1,t,19541


In [98]:
months

['feb', 'jan', 'dec', 'nov', 'oct', 'sep']

In [104]:
pd.read_sql_query('''
        select instant_bookable, count(*) as cnt
        from (
            select listing_id, min(price_usd) as price_per_night
            from calendar_feb
            where 1=1
                and date between '2019-02-08' and '2019-02-10'
            group by listing_id
            having max(available) == 't'
                and min(available) == 't'
            ) listings
        join instant inst
            on listings.listing_id = inst.id
        group by instant_bookable
        order by instant_bookable
    ''', engine)

,instant_bookable,cnt
0,f,4267
1,t,2366


In [139]:
20268/(20268+42416)

0.32333609852593964

In [13]:
instant_jun = pd.read_csv('http://data.insideairbnb.com/france/ile-de-france/paris/2018-06-07/data/listings.csv.gz', compression = 'gzip', low_memory=False)
instant_jun[['id', 'instant_bookable']].to_sql('instant_jun', con=engine, if_exists='replace')
del instant_jun
pd.read_sql_query('select instant_bookable, count(*) as cnt from instant_jun group by instant_bookable', con=engine)

,instant_bookable,cnt
0,f,42443
1,t,18086


In [15]:
18086/(18086+42443)

0.29879892283037884

In [20]:
instant_jul = pd.read_csv('http://data.insideairbnb.com/france/ile-de-france/paris/2018-07-08/data/listings.csv.gz', compression = 'gzip', low_memory=False)
instant_jul[['id', 'instant_bookable']].to_sql('instant_jul', con=engine, if_exists='replace')
del instant_jul
pd.read_sql_query('select instant_bookable, count(*) as cnt from instant_jul group by instant_bookable', con=engine)

,instant_bookable,cnt
0,f,43502
1,t,19346


In [21]:
19346/(19346+43502)

0.3078220468431772

In [18]:
instant_aug = pd.read_csv('http://data.insideairbnb.com/france/ile-de-france/paris/2018-08-13/data/listings.csv.gz', compression = 'gzip', low_memory=False)
instant_aug[['id', 'instant_bookable']].to_sql('instant_aug', con=engine, if_exists='replace')
del instant_aug
pd.read_sql_query('select instant_bookable, count(*) as cnt from instant_aug group by instant_bookable', con=engine)

,instant_bookable,cnt
0,f,44056
1,t,21977


In [19]:
21977/(21977+44056)

0.33281843926521587

In [17]:
instant_sep = pd.read_csv('http://data.insideairbnb.com/france/ile-de-france/paris/2018-09-10/data/listings.csv.gz', compression = 'gzip', low_memory=False)
instant_sep[['id', 'instant_bookable']].to_sql('instant_sep', con=engine, if_exists='replace')
del instant_sep
pd.read_sql_query('select instant_bookable, count(*) as cnt from instant_sep group by instant_bookable', con=engine)

,instant_bookable,cnt
0,f,42416
1,t,20268


In [139]:
20268/(20268+42416)

0.32333609852593964

In [134]:
instant_oct = pd.read_csv('http://data.insideairbnb.com/france/ile-de-france/paris/2018-10-09/data/listings.csv.gz', compression = 'gzip', low_memory=False)
instant_oct[['id', 'instant_bookable']].to_sql('instant_oct', con=engine, if_exists='replace')
del instant_oct
pd.read_sql_query('select instant_bookable, count(*) as cnt from instant_oct group by instant_bookable', con=engine)

,instant_bookable,cnt
0,f,41541
1,t,19682


In [140]:
19682/(19682+41541)

0.3214804893585744

In [135]:
instant_nov = pd.read_csv('http://data.insideairbnb.com/france/ile-de-france/paris/2018-11-06/data/listings.csv.gz', compression = 'gzip', low_memory=False)
instant_nov[['id', 'instant_bookable']].to_sql('instant_nov', con=engine, if_exists='replace')
del instant_nov
pd.read_sql_query('select instant_bookable, count(*) as cnt from instant_nov group by instant_bookable', con=engine)

,instant_bookable,cnt
0,f,40838
1,t,19519


In [141]:
19519/(19519+40838)

0.32339248140232285

In [136]:
instant_dec = pd.read_csv('http://data.insideairbnb.com/france/ile-de-france/paris/2018-12-07/data/listings.csv.gz', compression = 'gzip', low_memory=False)
instant_dec[['id', 'instant_bookable']].to_sql('instant_dec', con=engine, if_exists='replace')
del instant_dec
pd.read_sql_query('select instant_bookable, count(*) as cnt from instant_dec group by instant_bookable', con=engine)

,instant_bookable,cnt
0,f,39974
1,t,19907


In [142]:
19907/(19907+39974)

0.33244267797798965

In [137]:
instant_jan = pd.read_csv('http://data.insideairbnb.com/france/ile-de-france/paris/2019-01-13/data/listings.csv.gz', compression = 'gzip', low_memory=False)
instant_jan[['id', 'instant_bookable']].to_sql('instant_jan', con=engine, if_exists='replace')
del instant_jan
pd.read_sql_query('select instant_bookable, count(*) as cnt from instant_jan group by instant_bookable', con=engine)

,instant_bookable,cnt
0,f,39062
1,t,19297


In [143]:
19297/(19297+39062)

0.3306602237872479